In [1]:
import pandas as pd
import openpyxl
import firebase_admin
from firebase_admin import initialize_app
from firebase_admin import credentials, auth
from firebase_admin import db, firestore 
import datetime
import json
import ast

c:\Users\Comseven\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:109: RequestsDependencyWarning: urllib3 (2.0.3) or chardet (None)/charset_normalizer (2.1.1) doesn't match a supported version!
  warnings.warn(


In [2]:
cred = credentials.Certificate(r"D:\Workstuff\my-work-python-script\Database_in_pandas\Firebase_Shipment_Database\json_secret\logistec-firebase-adminsdk-mz58b-63b513fce5.json")

try:
    initialize_app(cred, {
        'databaseURL' : 'https://logistec-default-rtdb.asia-southeast1.firebasedatabase.app/'\
    })
except Exception:
    pass

firestore_db = firestore.client()

In [180]:
all_data = firestore_db.collection_group("สินค้าขาด-เกิน")#.where("PHYID", "==", "PHYID")
docs = all_data.stream()
products_doc = list(map(lambda x: x.to_dict(), docs))

filepath = r"D:\Workstuff\my-work-python-script\get_49return\test.xlsx"


df = pd.DataFrame(products_doc,).explode("Products")
#products_df = pd.DataFrame.from_records(, orient='keys')
#products_df
#seriesdf = pd.DataFrame(series)


In [181]:
number = df['ภาพประกอบ'].apply(lambda x: len(x) if isinstance(x, list) else 0).astype(int)
max_len = number.max()
column_name = {i : f'ภาพประกอบ {i+1}' for i in range(max_len)}

pic = pd.DataFrame(pd.Series(df["ภาพประกอบ"]).apply(pd.Series)).rename(columns=column_name)
no_product = df.drop(["ภาพประกอบ", "Products"], axis=1)
products = pd.json_normalize(products_doc, record_path=["Products"]).reindex(columns=["Product Code", "Product Name", "Serial", "Expected Number", "Scanned Number", "scannedTime"])

merged = pd.concat([no_product, pic],axis=1)
joined = pd.concat([products, merged.reset_index() ], axis=1).set_index(["PHYID", "Branch", "Doc Date", "ผู้สแกน", "index"])


In [182]:
with pd.ExcelWriter(filepath,mode="w", engine="xlsxwriter") as excel_file:
    joined.to_excel(excel_file, sheet_name="สินค้าครบ", header=True, index=True, merge_cells=False)